### Imports

In [19]:
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
import mysql.connector
import plotly.graph_objects as go
import os
import xgboost as xgb
import plotly.figure_factory as ff
import dash
from sklearn.preprocessing import MinMaxScaler
from dash import Dash, dcc, html, Input, Output 
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_score

### DB Connection


In [2]:
user = os.environ.get("DB_USER")
password = os.environ.get("DB_PASS")

In [3]:
connection = mysql.connector.connect(host = 'localhost',
                                    user = user,
                                    passwd = password,
                                    db = 'netflix')

In [4]:
# Making sure connection is there
connection

In [5]:
netflix_stock = pd.read_sql_query('''
SELECT
    * 
FROM
    netflix
''', connection)
netflix_stock

,Date,Open,High,Low,Close,Adj Close,Volume,year
0,5/23/2002,1.156429,1.242857,1.145714,1.196429,1.196429,104790000,2002
1,5/24/2002,1.214286,1.225000,1.197143,1.210000,1.210000,11104800,2002
2,5/28/2002,1.213571,1.232143,1.157143,1.157143,1.157143,6609400,2002
3,5/29/2002,1.164286,1.164286,1.085714,1.103571,1.103571,6757800,2002
4,5/30/2002,1.107857,1.107857,1.071429,1.071429,1.071429,10154200,2002
...,...,...,...,...,...,...,...,...
5030,5/16/2022,193.300003,195.889999,185.699997,186.509995,186.509995,16145000,2022
5031,5/17/2022,189.169998,191.399994,185.169998,190.559998,190.559998,9876700,2022
5032,5/18/2022,186.720001,187.699997,176.270004,177.190002,177.190002,9665600,2022
5033,5/19/2022,178.050003,186.300003,175.710007,183.479996,183.479996,10448500,2022


### Dash App

In [18]:
# Dash App
app = dash.Dash()
# Setup figure 1
diff_fig = px.line(
    netflix_stock,
    x='Date',
    y='Close')

# Annotations
diff_fig.add_annotation(x='1/16/2007', y='3.248571',
                   text='Netflix annouces that it will launch streaming video',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='1/17/2003', y='0.817143',
                   text='1 million subscribers',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='3/12/2008', y='4.862857',
                   text='Hulu Launches',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='9/15/2008', y='4.015714',
                   text='Market crash',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='9/22/2010', y='22.418571',
                   text='Netflix expands into Canada',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='9/19/2011', y='20.535713',
                   text='Netflix annouces that the dvd business will split off',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='11/25/2011', y='9.122857',
                   text='Netflix loses 800k subscribers',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='1/4/2012', y='11.492857',
                   text='Starts Europe expansion',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='2/1/2013', y='17.567142',
                   text='House of Cards starts streaming',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='6/24/2015', y='89.002853',
                   text='7:1 stock split',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='2/20/2020', y='386 ',
                   text='covid market crash',
                   showarrow=True,
                   arrowhead=1)
diff_fig.add_annotation(x='4/19/2022', y='348.609985',
                   text='loses subscribers for the first time in 10 years',
                   showarrow=True,
                   arrowhead=1)



# Start layout 
app.layout = html.Div([
    
    html.H1("Full Length Netflix Closing Price by Date",
            style={'textAlign': 'center'}),
     dcc.Graph(figure=diff_fig),
     html.Br(),
    
    html.H1("Netflix Stock Chart by Year",
           style={'textAlign': 'center'}),
    
    # Drop down menu 
    dcc.Dropdown(id='my_dropdown',
                 options=[
                     {'label': '2002', 'value': 2002},
                     {'label': '2003', 'value': 2003},
                     {'label': '2004', 'value': 2004},
                     {'label': '2005', 'value': 2005},
                     {'label': '2006', 'value': 2006},
                     {'label': '2007', 'value': 2007},
                     {'label': '2008', 'value': 2008},
                     {'label': '2009', 'value': 2009},
                     {'label': '2010', 'value': 2010},
                     {'label': '2011', 'value': 2011},
                     {'label': '2012', 'value': 2012},
                     {'label': '2013', 'value': 2013},
                     {'label': '2014', 'value': 2014},
                     {'label': '2015', 'value': 2015},
                     {'label': '2016', 'value': 2016},
                     {'label': '2017', 'value': 2017},
                     {'label': '2018', 'value': 2018},
                     {'label': '2019', 'value': 2019},
                     {'label': '2020', 'value': 2020},
                     {'label': '2021', 'value': 2021},
                     {'label': '2022', 'value': 2022}
                 ],
                placeholder="Select a year"),
                html.Div(id='output_container', children=[]),
                html.Br(),

                dcc.Graph(id='my_map', figure={}),
    
  
])

# Callbacks for input/output
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='my_map', component_property='figure')],
    [Input(component_id='my_dropdown', component_property='value')]
)

#updating graph based on selection 
def update_graph(option_slctd):
    container = "".format(option_slctd)
    dff = netflix_stock.copy()
    dff = dff[dff["year"] == option_slctd]

    fig = go.Figure()
    fig.add_trace(go.Candlestick(x = dff['Date'],
                                 open=dff['Open'],
                                 high=dff['High'],
                                 low=dff['Low'],
                                 close=dff['Close'],
                                 ))
    fig.update_layout(xaxis_rangeslider_visible=False)
   

    
    return container, fig

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [24/May/2022 12:22:56] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 12:22:57] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 12:22:57] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 12:22:57] "GET /_favicon.ico?v=2.3.1 HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 12:22:57] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 12:22:57] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 12:22:57] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [24/May/2022 12:22:57] "POST /_dash-update-component HTTP/1.1" 200 -


### Netflix Price Prediction

In [24]:
netflix_stock.shape

(5035, 8)

In [25]:
# get closing data
close_data = pd.read_sql_query('''
    SELECT
        Close
    FROM
        netflix
''', connection)
close_data.head()

,Close
0,1.196429
1,1.210000
2,1.157143
3,1.103571
4,1.071429


In [26]:
# Get closing numbers
data = netflix_stock[["Close"]]
# If price went up - 1, if price went down - 0
data["Target"] = netflix_stock.rolling(2).apply(lambda x: x.iloc[1] > x.iloc[0])["Close"]
data

,Close,Target
0,1.196429,NaN
1,1.210000,1.0
2,1.157143,0.0
3,1.103571,0.0
4,1.071429,0.0
...,...,...
5030,186.509995,0.0
5031,190.559998,1.0
5032,177.190002,0.0
5033,183.479996,1.0


In [27]:
netflix_stock_prev = netflix_stock.copy()
#shift data forward. Dont want the model to know the close price of the day we are trying to predict
netflix_stock_prev = netflix_stock_prev.shift(1)

In [28]:
# to make predictions i am going to need the Close, Volume, Open, High, and Low
predictors = ["Close", "Volume", "Open", "High", "Low"]
#merge our "x" with our predictors
data = data.merge(netflix_stock_prev[predictors]).iloc[1:]
data.head()

,Close,Target,Volume,Open,High,Low
1,1.210000,1.0,11104800.0,1.214286,1.225000,1.197143
2,1.157143,0.0,6609400.0,1.213571,1.232143,1.157143
3,1.103571,0.0,6757800.0,1.164286,1.164286,1.085714
4,1.071429,0.0,10154200.0,1.107857,1.107857,1.071429
5,1.076429,1.0,8464400.0,1.078571,1.078571,1.071429


In [29]:
# Random Forest Classifier Model
# 100 decison trees 
rfc_model = RandomForestClassifier(n_estimators=500, min_samples_split=200, random_state=10)

In [30]:
# backtesting
def rfc_backtest(data, rfc_model, predictors, start=1000, step=100):
    predictions = []
    # Loop over the dataset in increments
    for i in range(start, data.shape[0], step):
        # Split into train and test sets
        train = data.iloc[0:i].copy()
        test = data.iloc[i:(i+step)].copy()
        
        # Fit the random forest model
        rfc_model.fit(train[predictors], train["Target"])
        
        # Make predictions
        preds = rfc_model.predict_proba(test[predictors])[:,1]
        preds = pd.Series(preds, index=test.index)
        preds[preds > .6] = 1
        preds[preds<=.6] = 0
        
        # Combine predictions and test values
        combined = pd.concat({"Target": test["Target"],"Predictions": preds}, axis=1)
        
        predictions.append(combined)
    
    return pd.concat(predictions)


In [37]:
predictions = rfc_backtest(data, rfc_model, predictors)

In [38]:
precision_score(predictions["Target"], predictions["Predictions"])

0.5822550831792976